In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：使用基于示例的API定制训练图像分类模型进行在线预测，并附带解释

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/gapic/custom/showcase_custom_image_classification_online_explain_example_based_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/gapic/custom/showcase_custom_image_classification_online_explain_example_based_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebooks/community/gapic/custom/showcase_custom_image_classification_online_explain_example_based_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本笔记本演示了如何为您的模型获取基于示例的解释。解释可以帮助您回答关于为什么模型作出特定预测的问题，并将其与训练数据中的特征相关联。在这个演示中，我们将讨论以下内容：

1. 从Vertex Explainable AI服务中获取基于示例的解释。
2. 探索类似示例以理解模型预测的用例。

本笔记本的先决条件包括：
1. 一个预测模型和从中提取潜在表示（即嵌入）的方法。本笔记本将演示如何为深度神经网络执行此操作。
2. 一个Google Cloud项目。
3. 一个Google存储桶来托管模型和数据集。

一旦这些准备就绪，笔记本的三个主要部分将是：
1. 创建并上传启用解释的模型。
2. 创建一个`Endpoint`资源并将模型部署到其中。
3. 发出解释请求并检查它们。

### 数据集

在本笔记本中，我们将使用通过[TF Datasets](https://www.tensorflow.org/datasets/catalog/stl10)下载的[STL10数据集](https://cs.stanford.edu/~acoates/stl10/)。这个数据集是[ImageNet数据集](https://www.image-net.org/)的一个子集，只包含10个类别。

### 目标

本教程使用以下Google Cloud ML服务：

- `Vertex AI训练`
- `Vertex Explainable AI`
- `Vertex AI预测`
- `Vertex AI模型`

执行的步骤包括：

1. 准备训练数据
2. 微调图像分类模型以获取嵌入
3. 在Vertex AI模型注册表中注册模型
4. 将模型部署到Vertex AI Endpoint
5. 使用基于示例的解释API请求解释
6. 分析结果

### 成本

本教程使用Google Cloud的收费组件：

* Vertex AI
* Cloud Storage

了解有关[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)的信息，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期用量生成成本估算。

### 设置本地开发环境

**如果您正在使用Colab或Google Cloud笔记本，您可以跳过此步骤**，因为您的环境已经满足运行此笔记本的所有要求。

否则，请确保您的环境满足此笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用 Python 3 的虚拟环境中运行的Jupyter笔记本

Google Cloud指南[设置Python开发环境](https://cloud.google.com/python/setup) 和[Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了一套简明的说明：

1. [安装和初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端窗口中运行`pip3 install jupyter`命令。

5. 要启动Jupyter，请在终端窗口中运行`jupyter notebook`命令。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

if os.getenv("IS_TESTING"):
    ! pip3 install {USER_FLAG} --upgrade numpy tensorflow tensorflow_datasets -q
    ! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform -q
else:
    ! pip3 install {USER_FLAG} --upgrade numpy==1.21.6 tensorflow==2.8.0 tensorflow_datasets==4.6.0 -q
    ! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform==1.15.0 -q

### 重新启动内核

在安装/升级软件包之后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在开始之前

### 设置您的Google Cloud项目

**无论您使用哪种笔记本环境，都需要按照以下步骤操作。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您将获得$300的免费信用额度用于计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudresourcemanager.googleapis.com)。

1. 如果您在本地运行此笔记本，您将需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter会将以`！`开头的行作为shell命令运行，并将以`$`开头的Python变量插入这些命令。

**设置您的项目ID**

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，请在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project]"  # @param {type:"string"}

In [ ]:
! gcloud config set project $PROJECT_ID

获得您的项目编号

现在项目ID已设定，您将获得相应的项目编号。

In [ ]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

### 区域

您还可以更改 `REGION` 变量，该变量用于本笔记本中的操作。以下是 Vertex AI 支持的区域。我们建议您选择与您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶进行 Vertex AI 训练。并非所有区域都支持所有 Vertex AI 服务。

了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}

if REGION == "[your-region]":
    REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享的测试账户或项目。为了避免用户之间在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

验证您的 Google Cloud 帐号

**如果您正在使用 Vertex AI Workbench 笔记本电脑，您将跳过此步骤**，因为您的环境已经经过验证。

如果您正在使用Colab，请运行下面的单元格，并按提示进行OAuth身份验证。

否则，请按照以下步骤操作：

1. 在Cloud Console中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击 **创建服务帐号**。

3. 在**服务帐号名称**字段中输入一个名称，并点击 **创建**。

4. 在**将此服务帐号授权访问项目**部分，点击**角色**下拉列表。在过滤框中输入以下角色并选择

    - 服务帐号用户
    - 存储管理员
    - 存储对象管理员
    - Vertex AI管理员

5. 点击*创建*。将包含您密钥的JSON文件下载到本地环境。

6. 在下面的单元格中将您的服务帐号密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import sys

# If on Vertex AI Workbench, then don't execute this code

IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储存储桶

**无论您使用的是哪种笔记本环境，下面的步骤都是必需的。**

在下面设置您的云存储存储桶的名称。它必须在所有云存储存储桶中是唯一的。

您也可以更改`REGION`变量，该变量用于本笔记本后续操作。我们建议您选择一个[Vertex AI 服务可用的区域](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

运行以下单元格以创建您的云存储桶。如果该桶已经存在，您将收到一个错误，但这不会影响教程的其余部分。然而，您可能会在此桶中获得未期望的数据。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。如果这是一个新的桶，这个单元格不会产生输出。

In [ ]:
! gsutil ls -al $BUCKET_URI

获取您的服务账号

如果您不想使用项目的计算引擎服务账号，请将`SERVICE_ACCOUNT`设置为另一个服务账号ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### 设置服务账户访问权限

运行以下命令，向您在上一步中创建的存储桶授予服务账户访问权限。您只需要针对每个服务账户运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator {BUCKET_URI}

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer {BUCKET_URI}

创建本地目录 
接下来，您创建一些本地目录，用于本教程。

In [ ]:
DATA_PATH = "data"
DELIVERABLE_PATH = "deliverables"

! mkdir -m 777 -p {DATA_PATH}
! mkdir -m 777 -p {DELIVERABLE_PATH}

### 导入库

In [ ]:
import base64
import io
import json
# General
import time

# Training
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

import tensorflow as tf
import tensorflow_datasets as tfds

tfds.disable_progress_bar()
# Vertex AI
from google.cloud import aiplatform_v1beta1 as vertex_ai_v1beta1
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers

### 设置变量

#### 设定教程变量

为教程设定一些变量。

In [ ]:
# General
ENVIRON = "prod"
DATASET_NAME = (
    "stl10"  # Will be downloaded from https://www.tensorflow.org/datasets/catalog/stl10
)

# Model experimentation
RAW_DIR = f"{DATA_PATH}/raw/{DATASET_NAME}"
PREPROCESSED_DIR = f"{DATA_PATH}/preprocessed/{DATASET_NAME}"
MODEL_DIR = f"{DELIVERABLE_PATH}/models/mobilenetv2-{DATASET_NAME}"

设置Vertex AI常量

为Vertex设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的Vertex API服务端点。
- `PARENT`：用于数据集、模型、作业、流水线和端点资源的Vertex位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 设置硬件加速器

设置硬件加速器（例如，GPU）用于训练和预测。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`，以使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用带有4个Nvidia Telsa K80 GPU的GPU容器映像，您可以指定：

(aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

*注意*：在TF 2.3之前的GPU支持下发布的TF版本将无法在本教程中加载自定义模型。这是一个已知问题，在TF 2.3中已修复——这是由于在serving函数中生成的静态图操作而引起的。如果您在自定义模型中遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        vertex_ai_v1beta1.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (vertex_ai_v1beta1.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        vertex_ai_v1beta1.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

设置用于训练和预测的预先构建的Docker容器镜像。

获取最新列表，请查看[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

获取最新列表，请查看[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-8"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`以配置用于训练和预测的虚拟机的计算资源。
 - 机器类型
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存。
     - `n1-highcpu`：每个vCPU 0.9GB的内存。
 - vCPU数：\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下内容不支持用于训练*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2的机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### 辅助函数

以下是您在教程中使用的一系列辅助函数：

1. `create_index_to_name_map`：创建一个从索引到名称的映射，用于标签变量
2. `extract_images_and_labels`：从数据集中提取一批图像和标签的函数
3. `plot_input_and_neighbors`：绘制输入图像及其邻近图像的函数

In [ ]:
def create_index_to_name_map(ds_info):
    """
    Creates a map from label name to numerical index.
    Args:
        ds_info: DatasetInfo object.
    Returns:
        index_to_name_map: dict. Map from name to index.
    """
    index_to_name = {}
    num_classes = ds_info.features["label"].num_classes
    names = ds_info.features["label"].names
    for i in range(num_classes):
        index_to_name[i] = names[i]
    return index_to_name


def extract_images_and_labels(ds, num_batches):
    """
    Extract images and labels from a dataset.
    Args:
        ds: A dataset.
        num_batches: The number of batches to extract. -1 uses the whole dataset
    Returns:
        images: A numpy structure of images.
        labels: A numpy structure of labels.
    """
    data_slice = ds.take(num_batches)
    images = []
    labels = []
    for image, label in data_slice:
        images.append(image)
        labels.append(label)
    images = tf.concat(images, 0)
    labels = tf.concat(labels, 0)
    print(f"Image batch shape: {images.shape}")
    return images.numpy(), labels.numpy()


def plot_input_and_neighbors(
    val_img_idx,
    all_train_images,
    val_images,
    all_train_labels,
    val_labels,
    label_index_to_name,
    data_with_neighbors,
):
    """
    Plot the input image and its neighbors.
    Args:
        val_img_idx: Index of the input image.
        all_train_images: All training images.
        val_images: Validation images.
        all_train_labels: All training labels.
        val_labels: Validation labels.
        label_index_to_name: Dictionary mapping label indices to names.
        data_with_neighbors: Data with neighbors.
    Returns:
        None
    """
    image = val_images[val_img_idx]
    fig = plt.figure(figsize=(24, 12))
    ax_list = fig.subplots(3, 5)
    ax_list[0, 0].axis("off")
    ax_list[0, 1].axis("off")
    ax_list[0, 3].axis("off")
    ax_list[0, 4].axis("off")
    ax = ax_list[0, 2]
    class_label = val_labels[val_img_idx]
    ax.set_title(
        f"{class_label}:{label_index_to_name[class_label]} (example index: {val_img_idx})",
        fontsize=15,
    )
    ax.axis("off")
    ax.imshow(image.astype("uint8"))

    neighbor_list = data_with_neighbors[val_img_idx]["neighbors"]
    num_neighbors = len(neighbor_list)
    for n in range(num_neighbors):
        neighbor = neighbor_list[n]
        neighbor_idx = int(neighbor["neighborId"])
        neighbor_dist = neighbor["neighborDistance"]
        ax = ax_list[1 + n // 5, n % 5]
        class_label = all_train_labels[neighbor_idx]
        ax.set_title(
            f"{class_label}:{label_index_to_name[class_label]} (dist: {neighbor_dist:.3f})",
            fontsize=15,
        )
        ax.axis("off")
        ax.imshow(all_train_images[neighbor_idx].astype("uint8"))

# 教程

Vertex可解释的基于示例的API提供了一个高性能的ANN服务，用于返回类似于新预测/实例的示例。

为了利用基于示例的解释服务，您需要按照以下步骤进行操作：

1）对整个数据集建立索引：您需要提供一个嵌入模型在GCS存储桶中的路径，训练数据存储在GCS存储桶中，以及基于示例的解释的配置文件

2）部署索引和模型：您需要指定要使用的机器和在模型上传组中的模型标识符

3）查询类似的示例：您需要进行解释查询，模型将返回类似的示例

让我们开始为STL10数据集实验一个自定义模型，您将用它来提取嵌入。

实验

### 准备训练数据

下载并可视化数据

In [ ]:
split_ds, ds_info = tfds.load(
    DATASET_NAME,
    split=["train", "test"],
    as_supervised=True,  # Include labels
    with_info=True,
    shuffle_files=False,  # ensuring that the data doesn't get shuffled between runs
    data_dir=RAW_DIR,
)
train_ds, validation_ds = split_ds
tfds.show_examples(ds=train_ds, ds_info=ds_info);

In [ ]:
print(f'Number of classes in the dataset: {ds_info.features["label"].num_classes}')
print(f'Label names: {ds_info.features["label"].names}')
print(f'Number of examples in training split: {ds_info.splits["train"].num_examples}')
print(f'Number of examples in validation split: {ds_info.splits["test"].num_examples}')

准备图像

将图像缩放到下游模型所预期的大小。

In [ ]:
size = (224, 224)
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))

批量处理和预抓取数据

In [ ]:
batch_size = 32

train_ds = train_ds.batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.batch(batch_size).prefetch(buffer_size=10)

In [ ]:
label_index_to_name = create_index_to_name_map(ds_info)
print(label_index_to_name)

创建并训练模型

微调预训练分类模型的最后一层

您可以使用`MobileNetV2` [Keras Application](https://keras.io/api/applications/)深度学习模型，该模型可同时提供预训练权重，用于微调该模型以创建嵌入。

In [ ]:
# Each image can be flipped and rotated to generate more training data, and
# ensure the model is more robust to such changes, since a rotated "bird" should
# still be classified as a bird.

data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
size_3d = (224, 224, 3)
num_classes = ds_info.features["label"].num_classes

# Load an ImageNet model.
base_model = keras.applications.MobileNetV2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=size_3d,
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=size_3d)
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be normalized
# from (0, 255) to a range (-1., +1.), the normalization layer
# does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean**2

# Scale inputs to [-1, +1]
x = norm_layer(x)
norm_layer.set_weights([mean, var, 0])

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(num_classes)(x)
model = keras.Model(inputs, outputs)

model.summary()

在这个示范中，您可以通过解冻下层并微调来改善预训练模型的潜在表示。但是，您选择放弃这一步骤，以便学习到的表示与预训练模型完全相同，只对softmax层进行简短的训练。

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

epochs = 2
history = model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

### 评估模型

在训练后绘制模型的`准确率`和`损失率`。

In [ ]:
x_axis = range(1, epochs + 1)
plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(x_axis, history.history["sparse_categorical_accuracy"])
plt.plot(x_axis, history.history["val_sparse_categorical_accuracy"])
plt.title("Model Accuracy", fontsize=20)
plt.ylabel("Accuracy", fontsize=15)
plt.xlabel("Epoch", fontsize=15)
plt.legend(["Train", "Test"], fontsize=15)
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(x_axis, history.history["loss"])
plt.plot(x_axis, history.history["val_loss"])
plt.title("Model Loss", fontsize=20)
plt.ylabel("Loss", fontsize=15)
plt.xlabel("Epoch", fontsize=15)
plt.legend(["Train", "Test"], fontsize=15)
plt.grid();

正式化

现在您已经实验了模型以创建嵌入，让我们正式化训练以利用 Vertex AI。

### 设置客户端

Vertex AI客户端库以客户端/服务器模型运作。然后，您需要设置客户端以使用不同的服务。

在本教程中，您将针对工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 用于`模型`资源的模型服务。
- 用于部署的端点服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = vertex_ai_v1beta1.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = vertex_ai_v1beta1.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = vertex_ai_v1beta1.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = vertex_ai_v1beta1.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

训练模型

有两种方式可以使用容器镜像来训练一个自定义模型：

- **使用谷歌云预构建的容器**。如果使用预构建的容器，你需要额外指定一个要安装到容器镜像中的Python包。这个Python包包含了你用于训练自定义模型的代码。

- **使用自己的自定义容器镜像**。如果使用自己的容器，容器需要包含你用于训练自定义模型的代码。

### 准备您的自定义工作规范

现在您的客户已经准备好了，您的第一步是为自定义培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：指定用于培训的机器类型和数量（单个或分布式）
- `python_package_spec`：指定要与预构建容器一起安装的Python软件包。

#### 准备你的机器规格

现在为你的自定义训练作业定义机器规格。这告诉 Vertex AI 需要为训练提供什么类型的机器实例。
- `machine_type`: 要提供的 GCP 实例的类型 — 例如，n1-standard-8。
- `accelerator_type`: （如果有的话）硬件加速器的类型。在本教程中，如果之前设置变量 `TRAIN_GPU != None`，那么你正在使用 GPU；否则将使用 CPU。
- `accelerator_count`: 加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这告诉 Vertex 在每台机器实例上为训练提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD 或标准。SSD 更快，标准更便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘大小（单位为 GB）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

#### 定义工作池规范

接下来，您需要为自定义训练任务定义工作池规范。工作池规范将由以下内容组成：

- `replica_count`：要提供的该机器类型的实例数。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在 VM 实例上的 Python 训练包，以及要调用的 Python 模块，以及 Python 模块的命令行参数。

现在让我们深入了解 Python 包规范：

- `executor_image_spec`：这是为您的自定义训练任务配置的 Docker 镜像。

- `package_uris`：这是要安装在提供实例上的 Python 训练包的位置（URIs）列表。这些位置需要位于 Cloud Storage 存储桶中。这些可以是单个 Python 文件或整个包的 zip（归档）文件。在后一种情况下，作业服务将解压缩（解档）内容到 Docker 镜像中。

- `python_module`：用于运行自定义训练任务的 Python 模块（脚本）。在本例中，您将调用 `trainer.task.py` --请注意不需要附加 `.py` 后缀。

- `args`：要传递给相应的 Python 模块的命令行参数。在本例中，您将设置：
  - `"--model-dir=" + MODEL_URI`：存储模型工件的 Cloud Storage 位置。有两种方法可以告诉训练脚本保存模型工件的位置：
      - 直接：将 Cloud Storage 位置作为命令行参数传递给训练脚本（设置变量 `DIRECT = True`），
      - 间接：服务将 Cloud Storage 位置作为环境变量 `AIP_MODEL_DIR` 传递给训练脚本（设置变量 `DIRECT = False`）。在这种情况下，您在工作规范中告诉服务模型工件位置。
  - `"--epochs=" + EPOCHS`：训练的时代数。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单个或分布式训练的训练分发策略。
     - `"single"`：单个设备。
     - `"mirror"`：单个计算实例上的所有 GPU 设备。
     - `"multi"`：所有计算实例上的所有 GPU 设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_URI = f"{BUCKET_URI}/{DELIVERABLE_PATH}/models/mobilenetv2-{DATASET_NAME}"

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 2

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_URI,
        "--epochs=" + str(EPOCHS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_URI + "/trainer_stl10.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装工作规范

现在组装自定义工作规范的完整描述：

- `display_name`：您为这个自定义工作分配的可读名称。
- `job_spec`：自定义工作的规范。
    - `worker_pool_specs`：机器虚拟机实例的规范。
    - `base_output_directory`：这告诉服务在云存储中保存模型工件的位置（当变量 `DIRECT = False` 时）。然后，服务将该位置作为环境变量 `AIP_MODEL_DIR` 传递给训练脚本，路径格式为：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训包

#### 包布局

在开始培训之前，您将看一下如何为定制培训任务组装Python包。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 镜像的操作环境中的指令。

文件 `trainer/task.py` 是执行定制培训任务的 Python 脚本。*注意*，当我们在工作池规范中提到它时，我们会用点(`trainer.task`)替换目录斜杠，并且去掉文件后缀(`.py`)。

#### 包装配

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==4.0.1',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: STL10 image classification\n\nVersion: 0.0.0\n\nSummary: Demonstration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: googler@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 创建任务脚本

在下一个单元格中，你需要编写训练脚本 task.py 的内容。我们不会详细讨论，只是让你浏览一下。总结一下：

- 从命令行中获取保存模型 artifacts 的目录（`--model_dir`），如果没有指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets（tfds）中加载 STL10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型 (`compile()`)。
- 根据参数 `args.distribute` 设置训练分发策略。
- 根据参数 `args.epochs` 训练模型 (`fit()`)，设置 epochs 和 steps。
- 将训练好的模型保存到指定的模型目录中（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for STL10

# General
import os
import argparse
import sys
import logging

logging.basicConfig(format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')

# Training
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python.client import device_lib

tfds.disable_progress_bar()
from tensorflow import keras
from tensorflow.keras import layers

# Variables
N_CLASSES = 10
SIZE = (224, 224)
BUFFER_SIZE = 10
BATCH_SIZE = 32
SIZE_3D = (224, 224, 3)


# Helpers
# Get arguments
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model-dir', dest='model_dir',
                        default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
    parser.add_argument('--epochs', dest='epochs',
                        default=2, type=int,
                        help='Number of epochs.')
    parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                        help='distributed training strategy')
    args = parser.parse_args()
    return args


# Single Machine, single compute device
def get_strategy(distribute):
    if distribute == 'single':
        if tf.config.list_physical_devices('GPU'):
            strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
        else:
            strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")

    # Single Machine, multiple compute device
    elif distribute == 'mirror':
        strategy = tf.distribute.MirroredStrategy()

    # Multiple Machine, multiple compute device
    elif distribute == 'multi':
        strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
    return strategy


# Get train and validation datasets
def get_train_val_ds():
    split_ds = tfds.load(
        'stl10',
        split=["train", "test"],
        as_supervised=True,
        shuffle_files=False,
    )
    train_ds, validation_ds = split_ds
    return train_ds, validation_ds


# Get preprocessed dataset
def preprocess_dataset(train_ds, validation_ds):
    train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, SIZE), y))
    validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, SIZE), y))
    return train_ds, validation_ds


# Build the model
def build_and_compile_model():
    # Define data augmentation layers
    data_augmentation = keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
        ]
    )

    # Define base model
    base_model = keras.applications.MobileNetV2(
        weights="imagenet",
        input_shape=SIZE_3D,
        include_top=False,
    )

    # Freeze the base_model
    base_model.trainable = False

    # Create new model on top
    inputs = keras.Input(shape=SIZE_3D)
    x = data_augmentation(inputs)

    # Normalize inputs for Pre-trained Xception weights
    norm_layer = keras.layers.experimental.preprocessing.Normalization()
    mean = np.array([127.5] * 3)
    var = mean ** 2
    x = norm_layer(x)
    norm_layer.set_weights([mean, var, 0])

    # Set inference mode
    x = base_model(x, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
    outputs = keras.layers.Dense(N_CLASSES)(x)

    # Build the model
    model = keras.Model(inputs, outputs)
    model.summary()

    # Compile the model 
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

    return model


if __name__ == "__main__":

    # Initialize arguments and strategy
    logging.info('Initialize arguments and strategy')
    train_args = get_args()

    # Variables
    strategy = get_strategy(train_args.distribute)
    epochs = train_args.epochs
    model_dir = train_args.model_dir
    n_workers = strategy.num_replicas_in_sync
    global_batch_size = BATCH_SIZE * n_workers

    # Print versions
    logging.info('Python Version = {}'.format(sys.version))
    logging.info('TensorFlow Version = {}'.format(tf.__version__))
    logging.info('Python Version = {}'.format(sys.version))

    # Get TRAIN and VAL datasets
    logging.info('Get train and val datasets')
    train_ds, validation_ds = get_train_val_ds()

    # Preprocess TRAIN and VAL datasets
    logging.info('Get train and val datasets')
    train_ds, validation_ds = preprocess_dataset(train_ds, validation_ds)
    train_ds = train_ds.batch(global_batch_size).prefetch(buffer_size=BUFFER_SIZE)
    validation_ds = validation_ds.batch(global_batch_size).prefetch(buffer_size=BUFFER_SIZE)

    # Train model
    logging.info('Build model')
    with strategy.scope():
        model = build_and_compile_model()

    logging.info('Train model')
    model.fit(train_ds, epochs=epochs, validation_data=validation_ds)
    logging.info(f'Save model in {model_dir}')
    model.save(model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_stl10.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用这个辅助函数`create_custom_job`，它接受以下参数：

-`custom_job`：自定义作业的规范。

该辅助函数调用作业客户端服务的`create_custom_job`方法，带有以下参数：

-`parent`：指向`数据集`，`模型`和`终端`资源的Vertex位置路径。
-`custom_job`：自定义作业的规范。

您将显示`response`对象中返回的一些字段，其中最感兴趣的两个是：

`response.name`：分配给此自定义训练作业的Vertex完全限定标识符。您将保存此标识符以在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义工作的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

获取有关自定义工作的信息

接下来，使用这个辅助函数 `get_custom_job`，它接受以下参数：

- `name`：自定义工作的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的 `get_custom_job` 方法，带有以下参数：

- `name`：自定义工作的 Vertex 完全限定标识符。

如果您还记得，当调用 `create_custom_job` 方法时，在 `response.name` 字段中获取了自定义工作的 Vertex 完全限定标识符，并将标识符保存在变量 `job_id` 中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != vertex_ai_v1beta1.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == vertex_ai_v1beta1.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_URI
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

部署

### 加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在云存储存储桶中。现在从云存储存储桶加载它，然后您可以执行一些操作，如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras `model.load_model()` 方法，并向其传递保存模型的云存储路径 -- 由 `MODEL_DIR` 指定。

In [ ]:
model = tf.keras.models.load_model(MODEL_URI)

准备模型用于服务

接下来，您将把您的 TF.Keras 模型从自定义作业上传到 Vertex AI 的 `Model` 服务，这将为您的自定义模型创建一个 Vertex `Model` 资源。在上传过程中，您需要定义一个服务函数，将数据转换为模型所需的格式。如果您将编码后的数据发送到 Vertex AI，您的服务函数将确保在将数据作为输入传递给模型之前，在模型服务器上对数据进行解码。

服务函数的工作原理

当您向在线预测服务器发送请求时，该请求将通过 HTTP 服务器接收。HTTP 服务器从 HTTP 请求内容体中提取预测请求。提取的预测请求将转发到服务函数。对于 Google 预构建的预测容器，请求内容将作为 `tf.string` 传递给服务函数。

服务函数由两部分组成：

- `预处理函数`：
   - 将输入 (`tf.string`) 转换为底层模型的输入形状和数据类型（动态图）。
   - 执行与训练底层模型期间相同的数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`：
   - 将模型输出转换为接收应用程序期望的格式 -- 例如，压缩输出。
   - 为接收应用程序打包输出 -- 例如，添加标题，创建 JSON 对象等。

预处理和后处理函数均转换为静态图，然后与模型融合。底层模型的输出传递给后处理函数。后处理函数将转换/打包后的输出传回给 HTTP 服务器。HTTP 服务器将输出作为 HTTP 响应内容返回。

在为 TF.Keras 模型构建服务函数时需要考虑的一个因素是它们作为静态图运行。这意味着您不能使用需要动态图的 TF 图操作。如果您这样做，您将在服务函数的编译过程中收到一个错误，指示您正在使用不受支持的 EagerTensor。

#### 为图像数据创建服务函数

为了将图像传递给预测服务，您需要将压缩的（例如JPEG）图像字节编码为base 64格式 - 这样可以使内容在通过网络传输二进制数据时得到安全保护。由于这个部署的模型期望输入数据为原始（未压缩）字节，您需要确保base 64编码的数据在传递给部署的模型之前转换回原始字节。

为了解决这个问题，定义一个服务函数（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个`@tf.function`装饰器，使服务函数与底层模型融合在一起（而不是在CPU上游处理）。

当您发送预测或解释请求时，请求的内容会被base 64解码为一个Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数将`tf.string`预处理为原始（未压缩）的numpy字节（`preprocess_fn`），以匹配模型的输入要求：
- `io.decode_jpeg`- 解压缩JPG图像，返回一个具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为浮点32，并将像素数据重新缩放在0和1之间。
- `image.resize` - 调整图像的尺寸以匹配模型的输入形状。

此时，数据可以传递给模型（`m_call`）。

#### XAI 签名

当服务函数与底层模型一起保存（`tf.saved_model.save`）时，您需要指定服务函数的输入层为 `serving_default` 签名。

对于XAI图像模型，您需要保存两个额外的签名来自服务函数：

- `xai_preprocess`：服务函数中的预处理函数。
- `xai_model`：用于调用模型的具体函数。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    """
    The preprocess function.
    Args:
        bytes_input: The input image in bytes.
    Returns:
        The preprocessed image in numpy array.
    """
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    rescale = tf.cast(resized, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    """
    Preprocess the input image.
    Args:
        bytes_inputs: A list of raw image bytes.
    Returns:
        A list of preprocessed images.
    """
    decoded_images = tf.nest.map_structure(
        tf.stop_gradient, tf.map_fn(_preprocess, bytes_inputs, dtype=tf.float32)
    )
    return {CONCRETE_INPUT: decoded_images}


@tf.function(
    input_signature=[tf.TensorSpec([None], tf.string), tf.TensorSpec([None], tf.string)]
)
def serving_fn(id, bytes_inputs):
    """
    This function is used to serve the embeddings.
    Args:
        id: The id of the input.
        bytes_inputs: The input image.
    Returns:
        The output of the model.
    """
    images = preprocess_fn(bytes_inputs)
    embedding = m_call(**images)
    return {"id": id, "embedding": embedding}

提取并上传用于索引创建的嵌入模型

如上所述，您需要提供嵌入以便对数据集进行索引。在这种情况下，您可以跳过数据增强层并删除softmax层，以获取先前训练的模型中的嵌入。

In [ ]:
embedding_model = keras.Sequential()
for layer in model.layers[:-1]:  # go through until last layer
    print(layer.name)
    if "sequential" not in layer.name:  # skip data augmentation layer
        embedding_model.add(layer)
embedding_model.summary()
probability_model = keras.Sequential([model, tf.keras.layers.Softmax()])

In [ ]:
EMBEDDINGS_URI = (
    f"{BUCKET_URI}/{DELIVERABLE_PATH}/embeddings/mobilenetv2-{DATASET_NAME}"
)

m_call = tf.function(embedding_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    embedding_model,
    EMBEDDINGS_URI,
    signatures={
        "serving_default": serving_fn,
    },
)

获取服务功能签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称 -- 这在您进行预测请求时会用到。

您还需要知道服务功能的输入和输出层名称以构建解释元数据 -- 这将在后续讨论中详细讨论。

In [ ]:
embedding_model_loaded = tf.saved_model.load(EMBEDDINGS_URI)

serving_input = list(
    embedding_model_loaded.signatures["serving_default"]
    .structured_input_signature[1]
    .keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(
    embedding_model_loaded.signatures["serving_default"].structured_outputs.keys()
)[0]
print("Serving function output:", serving_output)

input_name = model.input.name
print("Model input name:", input_name)
output_name = model.output.name
print("Model output name:", output_name)

### 上传训练数据

接下来是上传训练数据。解释信息是从这些数据中提取的。你可以选择较少的批次以加快运行速度，但结果可能不够精确。

In [ ]:
if not os.path.exists(PREPROCESSED_DIR):
    os.makedirs(PREPROCESSED_DIR, exist_ok=True)

dataset_file = f"{DATASET_NAME}-train-images.jsonl"
saved_jsonl_path = f"{PREPROCESSED_DIR}/{dataset_file}"
input_tensor_name = "bytes_inputs"  # Must match the serving_fn definition

num_batches = -1  # uses the entire dataset
start = time.time()
all_train_images, all_train_labels = extract_images_and_labels(
    train_ds, num_batches=num_batches
)
end = time.time()
print(f"Time taken to process training data: {end - start:.5f} secs")

In [ ]:
start = time.time()
with open(saved_jsonl_path, "w") as f:
    for i, im in enumerate(all_train_images):
        img_bytes = io.BytesIO()
        image = Image.fromarray(im.astype(np.uint8))
        image.save(img_bytes, format="PNG")
        json.dump(
            {
                "id": str(i),
                "bytes_inputs": {
                    "b64": base64.b64encode(img_bytes.getvalue()).decode("utf-8")
                },
            },
            f,
        )
        f.write("\n")
! gsutil -m cp {saved_jsonl_path} {BUCKET_URI}
end = time.time()
print(f"Time taken to create and upload the training data: {end - start:.5f} secs")

基于示例的解释规范

最后，您需要定义基于示例的解释。在进行预测时获得解释，您必须在将自定义模型上传到 Vertex“模型”资源时启用解释功能并设置相应的设置。这些设置被称为解释元数据，包括：

- `parameters`：这是用于对模型进行解释的可解释性算法的规范。在本教程中，您将使用 `Examples`

- `metadata`：这是如何在您的自定义模型上应用算法的规范。

#### 解释参数

让我们首先深入了解解释性算法的设置。

#### 基于示例

基于示例的解释使得能够基于类比进行数据解释，适用于错误分析、模型调试以及新数据的批量标记。这可以带来更准确和健壮的模型，以及高效的数据标记流程。

参数：

- `examples`：这个参数允许定义返回提供数据集中最近邻的条件。

通过基于示例的解释，您可以获得一个带有相关参数配置的新解释方法。以下是您需要定义的主要属性列表。

- `dimensions`：嵌入的维度。
- `approximateNeighborsCount`：要返回的邻居数。
- `distanceMeasureType`：用于衡量示例相近程度的距离度量标准。您可以在``SQUARED_L2_DISTANCE、L1_DISTANCE、COSINE_DISTANCE 和 DOT_PRODUCT_DISTANCE``之间进行选择。
- `featureNormType`：对嵌入进行归一化，使其具有单位长度。您可以在``UNIT_L2_NORM 或 NONE``之间进行选择。
- `treeAhConfig`：控制近似质量和速度之间的权衡的参数。有关技术细节，请参阅论文。在幕后，它创建一个浅树，叶节点的数量由leafNodeEmbeddingCount控制，搜索召回/速度权衡由leafNodesToSearchPercent控制。

In [ ]:
DIMENSIONS = embedding_model.output.shape[1]
DATASET_FILE_PATH = f"{BUCKET_URI}/{dataset_file}"

NEAREST_NEIGHBOR_SEARCH_CONFIG = {
    "contentsDeltaUri": "",
    "config": {
        "dimensions": DIMENSIONS,
        "approximateNeighborsCount": 10,
        "distanceMeasureType": "SQUARED_L2_DISTANCE",
        "featureNormType": "NONE",
        "algorithmConfig": {
            "treeAhConfig": {
                "leafNodeEmbeddingCount": 1000,
                "leafNodesToSearchPercent": 100,
            }
        },
    },
}

NUM_NEIGHBORS_TO_RETURN = 10

EXAMPLES = vertex_ai_v1beta1.Examples(
    nearest_neighbor_search_config=NEAREST_NEIGHBOR_SEARCH_CONFIG,
    gcs_source=io_pb2.GcsSource(uris=[DATASET_FILE_PATH]),
    neighbor_count=NUM_NEIGHBORS_TO_RETURN,
)

PARAMETERS = vertex_ai_v1beta1.ExplanationParameters(examples=EXAMPLES)

#### 解释元数据

让我们首先深入了解解释元数据，它包括：

- `outputs`：它由输出名称到输出元数据的映射表示。在这种情况下，您期望得到嵌入。

- `inputs`：它由一个特征的输入的元数据表示。在这种情况下，您有编码图像和与之关联的ID。

In [ ]:
# for encoding parameter, 1 stands for 'IDENTITY'

EXPLANATION_INPUTS = {
    "my_input": vertex_ai_v1beta1.ExplanationMetadata.InputMetadata(
        {
            "input_tensor_name": input_tensor_name,
            "encoding": vertex_ai_v1beta1.ExplanationMetadata.InputMetadata.Encoding(1),
            "modality": "image",
        }
    ),
    "id": vertex_ai_v1beta1.ExplanationMetadata.InputMetadata(
        {
            "input_tensor_name": "id",
            "encoding": vertex_ai_v1beta1.ExplanationMetadata.InputMetadata.Encoding(1),
        }
    ),
}

EXPLANATION_OUTPUTS = {
    "embedding": vertex_ai_v1beta1.ExplanationMetadata.OutputMetadata(
        {"output_tensor_name": "embedding"}
    )
}

EXPLANATION_META_CONFIG = vertex_ai_v1beta1.ExplanationMetadata(
    inputs=EXPLANATION_INPUTS, outputs=EXPLANATION_OUTPUTS
)

EXPLANATION_SPEC = vertex_ai_v1beta1.ExplanationSpec(
    parameters=PARAMETERS, metadata=EXPLANATION_META_CONFIG
)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，以SavedModel格式存储，并将其上传到`Model`服务，该服务将实例化一个Vertex `Model`资源实例以用于您的模型。一旦完成，您可以像任何其他Vertex `Model`资源实例一样使用`Model`资源实例，例如部署到用于提供预测的`Endpoint`资源。

现在让我们深入研究Vertex模型规范`model`。这是一个包含以下字段的字典对象：

- `display_name`：`Model`资源的人类可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex `Dataset`资源的情况下构建的，因此您可以将其留空(`''`)。
- `artificat_uri`：存储嵌入向量的Cloud Storage路径，以SavedModel格式存储。
- `container_spec`：这是要安装在`Endpoint`资源上的Docker容器规范，`Model`资源将从中提供预测。使用之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则，只会分配CPU。
- `explanation_spec`：这是启用模型可解释性的规范。

辅助函数在一个配置中传递这些参数，并调用`Model`客户端服务的`upload_model`方法，该方法接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex根路径。
- `model`：Vertex `Model`资源实例的规范。

将模型上传到Vertex Model资源将返回一个长时间运行的操作，因为可能需要一些时间。您可以调用response.result()，这是一个同步调用，当Vertex Model资源准备就绪时将返回。

辅助函数返回相应的Vertex模型实例`upload_model_response.model`的完全限定标识符。您将保存此标识符，以便在后续步骤中使用变量`model_to_deploy_id`。

定义服务容器配置

In [ ]:
DEPLOY_IMAGE_URI = "gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-5:latest"

CONTAINER_CONFIG = {"image_uri": DEPLOY_IMAGE_URI}

CONTAINER_SPEC = vertex_ai_v1beta1.ModelContainerSpec(CONTAINER_CONFIG)

定义模型配置

In [ ]:
MODEL_NAME = f"similarity-{DATASET_NAME}-{TIMESTAMP}"

In [ ]:
MODEL_CONFIGURATION = {
    "display_name": MODEL_NAME,
    "artifact_uri": EMBEDDINGS_URI,
    "metadata_schema_uri": "",
    "container_spec": CONTAINER_SPEC,
    "explanation_spec": EXPLANATION_SPEC,
}

这个步骤可能需要长达一个小时的时间才能完成。目前，还没有简单的方法来监控这个进展。暴露运行日志和作业状态的功能计划在不久的将来（预览中）实现。

In [ ]:
def upload_model(model_configuration):

    model = vertex_ai_v1beta1.Model(
        display_name=model_configuration["display_name"],
        artifact_uri=model_configuration["artifact_uri"],
        metadata_schema_uri=model_configuration["metadata_schema_uri"],
        explanation_spec=model_configuration["explanation_spec"],
        container_spec=model_configuration["container_spec"],
    )

    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result()
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


uploaded_model_id = upload_model(MODEL_CONFIGURATION)

获取`Model`资源信息

现在让我们仅获取您的模型的模型信息。使用这个辅助函数`get_model`，带有以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

这个辅助函数调用Vertex `Model`客户端服务的方法`get_model`，带有以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(uploaded_model_id)

### 部署 `Model` 资源

现在部署训练好的 Vertex 定制 `Model` 资源。这需要两个步骤：

1. 为部署 `Model` 资源创建一个 `Endpoint` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源中。

#### 创建一个 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个用于部署模型以提供预测的端点，以下是参数：

- `display_name`：对于 `Endpoint` 资源的人类可读的名称。

辅助函数使用端点客户端服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`：对于 `Endpoint` 资源的人类可读的名称。

创建一个 `Endpoint` 资源会返回一个长时间运行的操作，因为可能需要一些时间来为服务提供资源进行预伟划。您调用 `response.result()`，这是一个同步调用，当端点资源准备就绪时将返回。辅助函数返回 `Endpoint` 资源的 Vertex 全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = f"similarity-{DATASET_NAME}-endpoint-{TIMESTAMP}"
DESCRIPTION = "An endpoint for the similarity model"
LABELS = {"env": ENVIRON, "status": "online"}


def create_endpoint(display_name, description, labels):
    endpoint = {
        "display_name": display_name,
        "description": description,
        "labels": labels,
    }
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result()
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME, DESCRIPTION, LABELS)

现在获得您创建的“Endpoint”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

#### 计算实例扩展

在处理在线预测请求时，您有几种选择来扩展计算实例：

- 单个实例：在线预测请求在单个计算实例上处理。
  - 将计算实例的最小(`MIN_NODES`)和最大(`MAX_NODES`)数量设置为一。

- 手动扩展：在线预测请求会分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小(`MIN_NODES`)和最大(`MAX_NODES`)数量设置为相同数量的节点。当模型首次部署到实例上时，固定数量的计算实例将被提供，并且在线预测请求会均匀分布在它们之间。

- 自动扩展：在线预测请求会分配到可扩展数量的计算实例上。
  - 将计算实例的最小(`MIN_NODES`)数量设置为在模型首次部署时提供的计算实例，并在负载情况下予以取消提供，并将最大(`MAX_NODES`)数量设置为根据负载条件提供的计算实例数量。

计算实例的最小数量对应于`min_replica_count`字段，最大数量对应于`max_replica_count`字段，这是在您的后续部署请求中的设置。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

#### 部署 `Model` 资源到 `Endpoint` 资源

使用这个辅助函数 `deploy_model` 将模型部署到您为提供预测创建的端点，使用以下参数：

- `model`: 要从训练流水线上传（部署）的 `Model` 资源的 Vertex 完全限定标识符。
- `deploy_model_display_name`: 部署模型的可读名称。
- `endpoint`: 要将 `Model` 资源部署到的 Vertex 完全限定的 `Endpoint` 资源标识符。

辅助函数调用 `Endpoint` 客户端服务的方法 `deploy_model`，该方法接受以下参数：

- `endpoint`: 要将 `Model` 资源部署到的 Vertex 完全限定的 `Endpoint` 资源标识符。
- `deployed_model`: 部署模型的要求。
- `traffic_split`: 端点中流向此模型的流量百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指代要上传的此模型，100 表示流量的 100%。
   - 如果端点上存在要分配流量的现有模型，则指定为，其中 `model_id` 是已部署到端点的模型的模型标识符。百分比必须加起来为 100。

           { "0": percent, model_id: percent, ... }

现在让我们更深入地了解 `deployed_model` 参数。该参数被指定为一个 Python 字典，具有最小所需字段：

- `model`: 要部署的（上传的） `Model` 资源的 Vertex 完全限定标识符。
- `display_name`: 部署模型的可读名称。
- `dedicated_resources`: 指的是为提供预测请求而扩展的计算实例（副本）数量。
  - `machine_spec`: 要预配的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
  - `min_replica_count`: 初始预配的计算实例数量，您之前设置为变量 `MIN_NODES`。
  - `max_replica_count`: 可扩展的计算实例最大数量，您之前设置为变量 `MAX_NODES`。
- `enable_container_logging`: 这启用容器事件的日志记录，例如执行失败（默认情况下禁用容器日志记录）。通常在调试部署时启用容器日志记录，然后在部署到生产中时禁用。

#### 流量分配

现在让我们更深入地了解 `traffic_split` 参数。该参数被指定为一个 Python 字典。一开始可能有点困惑。让我解释一下，您可以将模型的多个实例部署到一个端点，然后设置每个实例应该接收的流量百分比。

为什么要这样做呢？也许您已经在生产中部署了之前的版本 — 让我们称其为 v1。您在 v2 上得到了更好的模型评估，但在将其部署到生产环境之前，您无法确定它是否真的更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的端点，但只分配 10% 的流量给它。这样，您可以在不干扰大多数用户的情况下监控其表现，直到最终做出决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 同步等待操作完成，直到模型部署完毕。如果这是第一次将模型部署到端点，则可能需要额外几分钟来完成资源的预配。

In [ ]:
DEPLOYED_NAME = f"similarity-{DATASET_NAME}-deployed-{TIMESTAMP}"


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "enable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(uploaded_model_id, DEPLOYED_NAME, endpoint_id)

## 查询类似的例子

现在对部署的模型进行在线预测，使用验证数据集的样本来获取类似的例子。

准备验证数据
我们将为这些数据发布查询。为了演示目的，我们将选择完整验证数据集的一个小子集。

In [ ]:
val_dataset_file = f"{DATASET_NAME}-val-images.jsonl"
saved_val_jsonl_path = f"{PREPROCESSED_DIR}/{val_dataset_file}"

num_batches = 10
start = time.time()
val_images, val_labels = extract_images_and_labels(
    validation_ds, num_batches=num_batches
)
end = time.time()
print(f"Time taken to process validation data: {end - start:.5f} secs")

### 准备请求内容
您将以压缩的PNG图像形式发送STL10图像，而不是原始未压缩的字节。您还将字节编码为base 64——这样在通过网络传输二进制数据时，内容就不会被修改。您需要告诉服务二进制代码您的模型部署到了哪里，内容已经被base 64编码，以便它能在服务二进制代码的另一端解码。

预测请求中的每个实例都是以下形式的字典条目：

{`id`:, `bytes_inputs`: {'b64': content}}

- `id`：与图像相关联的唯一标识符。
- `bytes_inputs`：包含解码输入的映射。
- `'b64'`：指示内容已经以base 64编码的键。
- `content`：压缩的JPG图像字节，作为base 64编码的字符串。

In [ ]:
val_data = []

for i, im in enumerate(val_images):
    img_bytes = io.BytesIO()
    image = Image.fromarray(im.astype(np.uint8))
    image.save(img_bytes, format="PNG")
    instance = {
        "id": str(i),
        "bytes_inputs": {"b64": base64.b64encode(img_bytes.getvalue()).decode("utf-8")},
    }
    val_data.append(instance)

### 发送带有解释请求的预测

好的，现在你有了一个测试图片。使用这个辅助函数 `explain_image`，它接受以下参数：

- `image`: 作为 numpy 数组的测试图片数据列表。
- `endpoint`: `Model` 资源部署的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `parameters_dict`: 用于服务的额外参数。
- `deployed_model_id`: 当在端点上部署了多个模型时，部署模型的 Vertex 完全限定标识符。否则，如果只有一个模型被部署，可以设为 `None`。

这个函数使用预测客户端服务，并以以下参数调用 `explain` 方法：

- `endpoint`: `Model` 资源部署的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `instances`: 要预测和解释的实例（编码的图片）列表。
- `parameters`: 用于服务的额外参数。
- `deployed_model_id`: 当在端点上部署了多个模型时，部署模型的 Vertex 完全限定标识符。否则，如果只有一个模型被部署，可以设为 `None`。

由于 `predict()` 服务可以接受多个图片（实例），因此你将把单个图片作为一个图片列表发送。最后一步，你将实例列表打包成谷歌的 protobuf 格式 -- 这就是我们传递给 `explain()` 服务的内容。

`response` 对象返回一个列表，列表中的每个元素对应请求中的相应图片。你将在每个预测的输出中看到：

- `deployed_model_id` -- 执行预测/解释的模型的 Vertex 完全限定标识符。
- `predictions` -- 对十个类别的每个预测的置信水平 (`predictions`)，在 0 到 1 之间。
- `explanations` -- 每个特征如何影响预测。

In [ ]:
def explain_image(formatted_data, endpoint, parameters, deployed_model_id):

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = formatted_data
    instances = [
        json_format.ParseDict(instance, Value()) for instance in instances_list
    ]

    response = clients["prediction"].explain(
        endpoint=endpoint,
        instances=instances,
        parameters=parameters,
        deployed_model_id=deployed_model_id,
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)

    explanations = response.explanations
    print("explanations")
    for explanation in explanations:
        print(" explanation:", explanation)

    return response

In [ ]:
# Update input and reformat to match the expected schema.
BATCH_SIZE = (
    8  # The request payload has a size limit so we need to subbatch our request
)
NUM_VAL_DATA = 35

all_neighbors = []

for data_idx in range(0, NUM_VAL_DATA, BATCH_SIZE):
    end_idx = min(data_idx + BATCH_SIZE, NUM_VAL_DATA)
    formatted_data = val_data[data_idx:end_idx]
    response = explain_image(formatted_data, endpoint_id, None, deployed_model_id)
    all_neighbors = (
        all_neighbors + json_format.MessageToDict(response._pb)["explanations"]
    )

print(f"\nExamples processed: {len(all_neighbors)}")

### 保存输入ID和对应的邻居

对于您发送的每个输入图像，我们创建一个包含对应邻居的字典。

In [ ]:
# Save input ids and the corresponding neighbors
data_with_neighbors = []
input_data_list = val_data[:NUM_VAL_DATA]

for i, input_data in enumerate(input_data_list):
    neighbor_dict = all_neighbors[i]
    neighbor_dict["input"] = input_data["id"]
    data_with_neighbors.append(neighbor_dict)

DEBUG = False
if DEBUG:
    val_idx = 0
    print(data_with_neighbors[val_idx])
    print(data_with_neighbors[val_idx]["neighbors"])
    print(data_with_neighbors[val_idx]["input"])
    print(len(data_with_neighbors[val_idx]["neighbors"]))

### 用解释可视化图像

在以下表示中，您将看到每个发送图像的API生成的十个更接近的示例，根据您定义的距离。

正如您可以验证的那样，尽管“示例索引”结果接近于分类在同一类别的图像，但在某些情况下，模型错误地识别类别。您可以通过利用距离轻松可视化它们。

In [ ]:
val_img_indices = [1, 2, 10, 20, 25, 30]  # images to visually explore
for val_img_idx in val_img_indices:
    if val_img_idx > NUM_VAL_DATA - 1:
        raise ValueError(
            f"Data index {val_img_idx} does not exist in the requested explanations"
        )
    plot_input_and_neighbors(
        val_img_idx,
        all_train_images,
        val_images,
        all_train_labels,
        val_labels,
        label_index_to_name,
        data_with_neighbors,
    )

进一步探索
如果您想要继续探索，以下是一些想法：
1. 隔离模型出错的测试点（将猫误标为鸟），并可视化基于示例的解释，看看是否可以找到任何共同模式。
2. 如果通过这种分析，您发现您的训练数据缺少一些典型情况（猫的俯视图像），您可以尝试将这些图像添加到数据集中，看看是否可以提高模型性能。
3. [微调](https://keras.io/guides/transfer_learning/)模型的较低层，看看是否可以通过让模型学习更好的潜在表示来提高基于示例的解释的质量。

## 下线`Model`资源

现在从正在提供服务的`Endpoint`资源中下线您的`Model`资源。使用这个辅助函数`undeploy_model`，它接受以下参数：

- `deployed_model_id`：当`Model`资源部署到服务端点时，终端服务返回的模型部署标识符。
- `endpoint`：`Model`部署到的`Endpoint`资源的Vertex完全限定标识符。

此函数调用终端客户服务的`undeploy_model`方法，具有以下参数：

- `deployed_model_id`：当`Model`资源部署时，终端服务返回的模型部署标识符。
- `endpoint`：`Model`资源部署到的`Endpoint`资源的Vertex完全限定标识符。
- `traffic_split`：如何在`Endpoint`资源上分配流量给其余部署的模型。

由于这是在`Endpoint`资源上唯一部署的模型，您可以通过将其设置为空来简单地留下`traffic_split`。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理工作

要清理本项目中使用的所有GCP资源，您可以删除用于教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的单个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud存储桶

In [ ]:
delete_model = True
delete_endpoint = True
delete_customjob = True
delete_bucket = True

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=deployed_model_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete bucket
if delete_bucket and "BUCKET_URI" in globals():
    ! gsutil rm -r $BUCKET_URI

In [ ]:
! rm custom.tar.gz

In [ ]:
! rm -Rf {DATA_PATH} {DELIVERABLE_PATH} custom